In [2]:
import os

from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

## Uncomment the following files if you're not using pipenv as your virtual environment manager
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())


# Step 1: Setup LLM (Mistral with HuggingFace)
HF_TOKEN=os.environ.get("HF_TOKEN")
HUGGINGFACE_REPO_ID="mistralai/Mistral-7B-Instruct-v0.3"

def load_llm(huggingface_repo_id):
    llm=HuggingFaceEndpoint(
        repo_id=huggingface_repo_id,
        temperature=0.5,
        model_kwargs={"token":HF_TOKEN,
                      "max_length":"512"}
    )
    return llm

# Step 2: Connect LLM with FAISS and Create chain

CUSTOM_PROMPT_TEMPLATE = """
Use the pieces of information provided in the context to answer user's question.
If you dont know the answer, just say that you dont know, dont try to make up an answer. 
Dont provide anything out of the given context

Context: {context}
Question: {question}

Start the answer directly. No small talk please.
"""

def set_custom_prompt(custom_prompt_template):
    prompt=PromptTemplate(template=custom_prompt_template, input_variables=["context", "question"])
    return prompt

# Load Database
DB_FAISS_PATH="vectorstore/db_faiss"
embedding_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db=FAISS.load_local(DB_FAISS_PATH, embedding_model, allow_dangerous_deserialization=True)

# Create QA chain
qa_chain=RetrievalQA.from_chain_type(
    llm=load_llm(HUGGINGFACE_REPO_ID),
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={'k':3}),
    return_source_documents=True,
    chain_type_kwargs={'prompt':set_custom_prompt(CUSTOM_PROMPT_TEMPLATE)}
)

# Now invoke with a single query
user_query=input("Write Query Here: ")
response=qa_chain.invoke({'query': user_query})
print("RESULT: ", response["result"])
print("SOURCE DOCUMENTS: ", response["source_documents"])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Write Query Here:  causes of fybrosis


D:\apk\conda\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


RESULT:  
Causes of fibrosis (Fibromyalgia) are not known exactly, but it is more frequent in women ages 20–50.
SOURCE DOCUMENTS:  [Document(id='aa90a00b-7795-4d17-86a3-1e43bd25d3a3', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2010-12-22T13:15:56+00:00', 'author': 'Tierney, Lawrence M.; Saint, Sanjay.; Whooley, Mary A.', 'keywords': '0071637907\r\n9780071637909', 'moddate': '2018-08-02T09:28:12+05:30', 'subject': 'McGraw-Hill Medical', 'title': 'Current Essentials of Medicine', 'source': 'data\\Current Essentials of Medicine(1)(1).pdf', 'total_pages': 607, 'page': 66, 'page_label': '54'}, page_content='54 Current Essentials of Medicine\n2\nIdiopathic Pulmonary Fibrosis\n■ Essentials of Diagnosis\n• Insidious onset of exertional dyspnea and dry cough in patients,\nusually in their sixth or seventh decades\n• Deﬁnition requires the histopathologic pattern of usual interstitial\npneumonia\n• Inspiratory crackles by auscultation; clubbing\n• Hypoxemia, especially e